In [ ]:
import os
import re
import json
import warnings
import huggingface_hub

from langchain.schema import Document
from langchain_community.chat_models import ChatOllama, ChatOpenAI

from langchain_experimental.text_splitter import SemanticChunker
from langchain_text_splitters import RecursiveCharacterTextSplitter

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.callbacks.manager import CallbackManager
from langchain_core.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

from langchain_openai import OpenAIEmbeddings
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

from langchain_upstage import UpstageEmbeddings
from langchain_community.embeddings import OllamaEmbeddings
from langchain.retrievers import ContextualCompressionRetriever
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from langchain.retrievers.document_compressors import CrossEncoderReranker

os.environ["TOKENIZERS_PARALLELISM"] = "false"
warnings.filterwarnings("ignore", category=FutureWarning)

from dotenv import load_dotenv
load_dotenv("../keys.env")

upstage_api_key = os.getenv("UPSTAGE_API_KEY")
os.environ['UPSTAGE_API_KEY'] = upstage_api_key

openai_api_key = os.getenv('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = openai_api_key

hf_token = os.getenv("HF_TOKEN")
huggingface_hub.login(hf_token)

from tqdm import tqdm
from openai import OpenAI

In [2]:
class Args:
    retrieval_debug = False
    llm_model = "ollama"
    
    src_lang = "ko"
    if src_lang == "en":
        eval_file_path = "../dataset/eval.jsonl" ## "../dataset/en_eval.jsonl" --> 성능이 별로임.
        doc_file_path = "../dataset/en_4.0_document.jsonl" ## "../dataset/processed_documents.jsonl"
    else:
        eval_file_path = "../dataset/eval.jsonl"
        doc_file_path = "../dataset/processed_documents.jsonl"

    output_path = "./outputs/output.csv"

    ## sparse or dense or ensemble
    doc_method = "dense"
    encoder_method = "huggingface" ## huggingface, upstage, openai
    retriever_weights = [0.6, 0.4] ## [sparse, dense]

    ## HuggingFace
    hf_model_name = "intfloat/multilingual-e5-large-instruct"
    model_kwargs = {"device": "cuda:0"}
    encode_kwargs = {"normalize_embeddings": False,
                     "clean_up_tokenization_spaces": True}
    
    ## Upstage
    upstage_model_name = "solar-embedding-1-large-passage"
    faiss_index_file = "./index_files/upstage-faiss.npy"
    
    ## OpenAI
    openai_model_name = "text-embedding-3-large"

    ## chunking
    chunking = True
    chunk_method = "recursive" ## recursive, semantic
    semantic_chunk_method = "upstage"
    chunk_size = 320
    chunk_overlap = 80

    ## query expension
    query_expansion = False

    ## query ensemble
    query_ensemble = False  # 쿼리 앙상블 수행 여부
    # 앙상블에 사용할 모델
    ensemble_models = [
        # {'type': 'hf', 'name': ""},
        # {'type': 'hf', 'name': "nlpai-lab/KoE5"},
        # {'type': 'hf', 'name': "BAAI/bge-large-en-v1.5",
        # {'type': 'hf', 'name': "intfloat/multilingual-e5-large"},
        # {'type': 'upstage', 'name': "solar-embedding-1-large-query"},
        # {'type': 'hf', 'name': "sentence-transformers/all-MiniLM-L6-v2"},
    ]
    ensemble_weights = [1]  # 각각의 모델 가중치 설정

    ## reranker
    rerank = False
    reranker_name = "BAAI/bge-reranker-v2-m3"  ## "BAAI/bge-reranker-large"

In [3]:
def load_ollama_encoder(model_name):
    encoder = OllamaEmbeddings(model_name)

    return encoder

def load_upstage_encoder(model_name):
    encoder = UpstageEmbeddings(model=model_name)

    return encoder

def load_openai_encoder(model_name):
    encoder = OpenAIEmbeddings(model=model_name)

    return encoder

def load_hf_encoder(model_name, model_kwargs, encode_kwargs):
    encoder = HuggingFaceEmbeddings(model_name=model_name,
                                    model_kwargs=model_kwargs,
                                    encode_kwargs=encode_kwargs)
    
    return encoder

def load_hf_reranker(model_name, retriever):
    reranker = HuggingFaceCrossEncoder(model_name=model_name)
    compressor = CrossEncoderReranker(model=reranker, top_n=3)
    compression_retriever = ContextualCompressionRetriever(base_compressor=compressor, base_retriever=retriever.as_retriever(search_kwargs={"k": 10}))

    return compression_retriever

In [4]:
def load_jsonl(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return [json.loads(line) for line in f]
    
def load_document(path='../dataset/documents.jsonl'):
    raw_documents = load_jsonl(path)

    documents = []
    for doc in raw_documents:
        doc_id = doc['docid']
        content = doc['content']
        documents.append(Document(page_content=content, metadata={"docid": doc_id}))

    return documents

def chunking(args, documents):
    if args.chunk_method == "recursive":
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=args.chunk_size,
            chunk_overlap=args.chunk_overlap,
            length_function=len,
            is_separator_regex=False
        )
    elif args.chunk_method == "semantic":
        if args.semantic_chunk_method == "huggingface":
            encoder = load_hf_encoder(args.hf_model_name, args.model_kwargs, args.encode_kwargs)
        elif args.semantic_chunk_method == "upstage":
            encoder = load_upstage_encoder(args.upstage_model_name)
        elif args.semantic_chunk_method == "openai":
            encoder = load_openai_encoder(args.openai_model_name)

        text_splitter = SemanticChunker(encoder)

    return text_splitter.split_documents(documents)

In [ ]:
args = Args()

documents = load_document(path=args.doc_file_path)
print(len(documents))

documents = chunking(args, documents)
print(len(documents))

In [6]:
def ollama_contextual_retrieval(model):
    prompt_context3 = (
    "입력된 문서 조각을 읽고 핵심을 파악해서 문서조각으로부터 '제목', '요약', '본문', '인사이트', '관련 질문들'을 만들어내고 위키피디아 형식으로 작성해주세요.\n"
    "단, '*'나 '#'등의 특수 기호는 사용하면 안됩니다."

    "입력 : {document_chunk}\n"
    "출력 :"

    )

    prompt3 = ChatPromptTemplate.from_template(prompt_context3)
    chain3 = prompt3 | model | StrOutputParser()

    return chain3

In [7]:
model = ChatOllama(model="llama3.2-ko", temperature=0.1)
chain3 = ollama_contextual_retrieval(model)

In [ ]:
with open('../dataset/contextual_retrieval_documents.jsonl', 'w', encoding='utf-8') as f:
    # for document in tqdm(documents):
    for document in documents:
        print(document.metadata['docid'])
        print(document.page_content)

        result = chain3.invoke({"document_chunk" : document.page_content})
        print(f"{result}\n")

        result_with_id = {
            "docid": document.metadata['docid'],
            "content": result
        }
        f.write(json.dumps(result_with_id, ensure_ascii=False) + '\n')